In [1]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['belka']  # Replace with your database name
collection = db['train_metadata']  # Replace with your collection name


# Using the aggregation framework to get N random documents
random_documents = list(collection.aggregate([
    {'$project': {'molecule_smiles': 1, 'binds': 1, 'protein_name':1,'_id': 0 ,'test_u':1, 'test1':1, 'test2':1}}]))

: 

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

/home/anonymous/miniconda3/envs/belka/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
random_documents = pd.DataFrame(random_documents)
unique_smiles = random_documents['molecule_smiles'].unique()

In [ ]:
db = client['belka']  # Replace with your database name
collection = db['full_molecules']  # Replace with your collection name

# The field you want to retrieve, and assuming '_id' is not needed
projection = {'Avalon': 1, 'Torsion':1, 'RDKFingerprint':1, 'maccs':1, 'ecfp':1, 'molecular_size':1, 'molecule_smiles':1, '_id': 0}  # Adjust 'needed_field' to your specific field name
# Define batch size
batch_size = 10000
current_skip = 0
documents = []

# Properly manage range for unique_smiles slicing
steps = len(unique_smiles) // batch_size + (1 if len(unique_smiles) % batch_size != 0 else 0)

# Using tqdm to iterate over the batches
for i in tqdm(range(steps)):
    start_index = i * batch_size
    end_index = start_index + batch_size
    batch_query = unique_smiles[start_index:end_index]
    batch_docs = list(collection.find({'molecule_smiles': {'$in': list(batch_query)}}, projection))
    if not batch_docs:  # Check if the fetched batch is empty
        break
    documents.extend(batch_docs)

# Convert results to a Pandas DataFrame
df = pd.DataFrame(documents)

100%|██████████| 199/199 [11:39<00:00,  3.51s/it]


In [ ]:
del documents

In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import OneHotEncoder

# Step 1: Merge DataFrames
merged_df = pd.merge(random_documents, df, on='molecule_smiles', how='inner')

# Step 2: One-Hot Encoding for protein_name
encoder = OneHotEncoder(sparse=False)
one_hot_encoded = encoder.fit_transform(merged_df[['protein_name']])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['protein_name']))
merged_df = pd.concat([merged_df, one_hot_df], axis=1).drop('protein_name', axis=1)


/home/anonymous/miniconda3/envs/belka/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
merged_df.to_parquet('training_all_fingers_set.parquet', engine='pyarrow')